In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
file1 = pd.read_csv("/content/drive/MyDrive/Ml_assignment1/student-mat.csv", sep=";")
file2 = pd.read_csv("/content/drive/MyDrive/Ml_assignment1/student-por.csv", sep=";")

In [ ]:
file1["subject"] = "maths"
file2["subject"] = "portugal"

In [ ]:
file3 = pd.concat([file1, file2], ignore_index=True)


file3.to_csv("merged.csv", index=False, sep=";")

In [ ]:
file3

In [ ]:
# pip install ucimlrepo

In [ ]:
X = file3.drop(columns=["G1", "G2", "G3"])
Y = file3['G3']


In [ ]:
string_features = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic','subject']

In [ ]:
Y

In [ ]:
encoded_X =X.copy()

for i in string_features:
  encoded_X[i]=pd.factorize(encoded_X[i])[0]




In [ ]:
encoded_X

In [ ]:
y=Y.values
type(y)

####Convert all the values in the G3 attribute to 0 (bad) if the value is less than 10, to 1 (good) if the value is between 10 and 15 (both including), and to 2 (great) if the value is greater than 15. Normalize all the other attributes by Z-score normalization, and segregate them into 4 equally spaced bins each giving the values between [0 to 3], and replace the values for that attribute with the number corresponding to the interval they belong.


In [ ]:
# y.value_counts()
y


In [ ]:

for i in range(len(y)):
  if y[i]<10:
    y[i]=0
  elif y[i]>=10 and y[i]<=15:
    y[i]=1
  else:
    y[i]=2

In [ ]:
print(y)

In [ ]:
one =0
zero=0
two=0
for i in y:
  if i==1:
    one+=1
  elif i==2:
    two+=1
  else:
    zero+=1
print(" 0 :",zero)
print(" 1 :",one)
print(" 2 :",two)

In [ ]:
print(X.columns)

In [ ]:
feature=[]
for i in X:
  # print(i)
  feature.append(i)
print(feature)

In [ ]:
data=[]
for i in range(len(feature)):
  xi=encoded_X[feature[i]].values
  data.append(xi)

In [ ]:
len(data)

In [ ]:
data[2]

In [ ]:
for i in  range(len(data)):
  # if(feature[i] in string_features):
  #   continue
  mean_val=data[i].mean()
  std_val=data[i].std()
  for j in range(len(data[i])):
    data[i][j]=(data[i][j]-mean_val)/std_val

In [ ]:
print(data[2])

In [ ]:
len(data),len(feature)

In [ ]:
data=np.array(data).T

In [ ]:
len(data[0])

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
final_data= df.copy()
for j in df.columns:
    final_data[j] = pd.cut( df[j] , labels=[0, 1, 2, 3] , bins=4, include_lowest=True)

In [ ]:
df

In [ ]:
final_data

#####Implement the standard ID3 Decision Tree algorithm, using both a) information gain, and b) gini index to choose which attribute to split at each point, and discuss the differences between these two criteria. Also, explain which of the two criteria you would choose and why. Stop splitting a node if it has less than 10 data points. Do NOT use scikit-learn for this part.


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(final_data, y, test_size=0.2, random_state=42)
print(len(x_train),len(y_train),len(x_test),len(y_test))


In [ ]:
def entropy(y):
  d={}
  for i in y:
    if i in d:
      d[i]+=1
    else:
      d[i]=1

  e=0
  n=len(y)
  for i in d:
    if d[i] > 0:
      e+=-(d[i]/n)* (np.log2(d[i]/n))
  return e



def info_gain(x, y, target_f):
    ep = entropy(y)
    wp = 0
    uniquev = x[target_f].unique()
    for i in uniquev:
        j_index = (x[target_f] == i)
        v = y.loc[j_index]
        p = len(v) / len(y)
        wp += p * entropy(v)
    return ep - wp

In [ ]:
def gini(y):
    d = {}
    for i in y:
        if i in d:
            d[i] += 1
        else:
            d[i] = 1

    g = 1
    n = len(y)
    for i in d:
        p = d[i] / n
        g =g- p * p
    return g


def gini_gain(x, y, target_f):
    gp = gini(y)   # parent gini
    wp = 0
    uniquev = x[target_f].unique()

    for i in uniquev:
        j_index = (x[target_f] == i)
        v = y.loc[j_index]
        p = len(v) / len(y)
        wp += p * gini(v)

    return gp - wp


In [ ]:
def majority(y):
    d={}
    for i in y:
      if i in d:
        d[i]+=1
      else:
        d[i]=1

    return max(d, key=d.get)

In [ ]:
def decision_tree_infogain(x, y):

    x = x.reset_index(drop=True)
    y = y.reset_index(drop=True)

    if len(y) < 10:
        return majority(y)

    if len(x.columns) == 0:
        return majority(y)

    if len(y.unique()) == 1:
        return y.iloc[0]

    gain=[]
    for i in x.columns:
      gain.append(info_gain(x,y,i))

    bf = x.columns[np.argmax(gain)]

    if np.max(gain) == 0:
        return majority(y)

    dtree = {bf: {}}

    u=x[bf].unique()
    for i in u:
        indexl = (x[bf] == i)
        listx = x.loc[indexl].drop(columns=[bf])

        listy = y.loc[indexl]

        dtree[bf][i] = decision_tree_infogain(listx, listy)

    return dtree

<!-- {14: {0: {np.int64(6): {2: {np.int64(20): {0: {np.int64(30): {3: {np.int64(0): {0: {np.int64(23): {1: {np.int64(13): {0: np.int64(1), 1: 1, 2: 1, 3: 2}}, 2: {np.int64(17): {0: {np.int64(10): {1: {np.int64(7): {2: {np.int64(28): -->

In [ ]:
def decision_tree_ginigain(x, y):

    x = x.reset_index(drop=True)
    y = y.reset_index(drop=True)

    if len(y) < 10:
        return majority(y)

    if len(x.columns) == 0:
        return majority(y)

    if len(y.unique()) == 1:
        return y.iloc[0]

    gain=[]
    for i in x.columns:
      gain.append(gini_gain(x,y,i))

    bf = x.columns[np.argmax(gain)]

    if np.max(gain) == 0:
        return majority(y)

    dtree = {bf: {}}

    u=x[bf].unique()
    for i in u:
        indexl = (x[bf] == i)
        listx = x.loc[indexl].drop(columns=[bf])

        listy = y.loc[indexl].reset_index(drop=True)

        dtree[bf][i] = decision_tree_ginigain(listx, listy)

    return dtree

In [ ]:
def predict1(data, tree):

    if not isinstance(tree, dict):
        return tree

    feature = list(tree.keys())[0]

    tvalue = data[feature]

    if tvalue in tree[feature]:
        return predict1(data, tree[feature][tvalue])
    else:
        return None


In [ ]:
def findaccuracy(x, y, tree):
    pred = []

    for temp , row in x.iterrows():
        s = predict1(row, tree)
        pred.append(s)

    correct = 0

    for p, t in zip(pred, y):
        if p == t:
            correct += 1
    return correct / len(y)


In [ ]:
def allprediction(x, tree):
    pred = []
    for temp, row in x.iterrows():
        s = predict1(row, tree)
        pred.append(s)
    return pred

def prec_rec(x, y, tree):
    pred = allprediction(x, tree)

    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for p, t in zip(pred, y):
        if p == 1 and t == 1:
            tp += 1
        elif p == 1 and t == 0:
            fp += 1
        elif p == 0 and t == 1:
            fn += 1


    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    a=[]
    a.append(precision)
    a.append(recall)
    return a


In [ ]:
y_train2=pd.Series(y_train)
dtree = decision_tree_infogain(x_train, y_train2)
print(dtree)

In [ ]:
tree  = decision_tree_infogain(x_train, y_train2)
accuracy = findaccuracy(x_test, y_test, tree)
a=prec_rec(x_test, y_test, tree)
print(f"With Information Gain Accuracy   : {accuracy*100:.4f}%")
print(f"With Information Gain precision  : {a[0]*100:.4f}%")
print(f"With Information Gain recall     : {a[1]*100:.4f}%")



In [ ]:
tree  = decision_tree_ginigain(x_train, y_train2)
accuracy = findaccuracy(x_test, y_test, tree)
a=prec_rec(x_test, y_test, tree)

print(f"With Gini Gain Accuracy   : {accuracy*100:.4f}%")
print(f"With Gini Gain precision  : {a[0]*100:.4f}%")
print(f"With Gini Gain recall     : {a[1]*100:.4f}%")

Test out the implementation of Decision Tree Classifier from the scikit-learn package, using a)
information gain, b) gini index, and discuss the differences between these two criteria. Also, explain
which of the two criteria you would choose and why. This page provides the documentation for
implementing decision tree from the scikit-learn package using the gini, and entropy (Shannon
information gain) criteria. Here, also stop splitting a node if it has less than 10 data points.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model1 = DecisionTreeClassifier(criterion = "entropy",min_samples_split = 10,random_state = 42)
model1.fit(x_train, y_train)

y_pred = model1.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy with Info gain:", acc*100)



In [ ]:
model2 = DecisionTreeClassifier(criterion = "gini",min_samples_split = 10,random_state = 42)
model2.fit(x_train, y_train)

y_pred2 = model2.predict(x_test)
acc2 = accuracy_score(y_test, y_pred2)
print("Accuracy with Gini Gain:", acc2*100)


Cross-validate the classifiers with 3-folds and print the mean macro accuracy, macro precision, and
macro recall for both the classifiers. You may or may not use the scikit-learn implementations
for computing these metrics and cross-validation.


In [ ]:
from sklearn.model_selection import KFold
def k_fold(k,data,y):
  kf=KFold(n_splits=k,shuffle=True)
  return kf

x=k_fold(3,final_data,y)
x

In [ ]:
# for info gain
accuracy=[]
p=[]
r=[]
sum=0
i=1
for itrain,itest in x.split(final_data):
  x_train  = data[itrain]
  x_test = data[itest]
  y_train = y[itrain]
  y_test =  y[itest]

  x_train2=pd.DataFrame(x_train)
  x_test2=pd.DataFrame(x_test)
  y_train2=pd.Series(y_train)

  tree  = decision_tree_infogain(x_train2, y_train2)
  acc = findaccuracy(x_test2, y_test, tree)
  accuracy.append(acc)
  print(f"Accuracy in fold{i} : ",acc*100)
  a =prec_rec(x_test2, y_test, tree)
  p.append(a[0])
  r.append(a[1])

  i+=1

print()

print("Mean accuracy  : ",np.mean(accuracy)*100)
print("Mean precision : ",np.mean(p)*100)
print("Mean recall    : ",np.mean(r)*100)


In [ ]:
# for gini gain
accuracy=[]
p=[]
r=[]
sum=0
i=1
for itrain,itest in x.split(final_data):
  x_train  = data[itrain]
  x_test = data[itest]
  y_train = y[itrain]
  y_test =  y[itest]

  x_train2=pd.DataFrame(x_train)
  x_test2=pd.DataFrame(x_test)
  y_train2=pd.Series(y_train)

  tree  = decision_tree_ginigain(x_train2, y_train2)
  acc = findaccuracy(x_test2, y_test, tree)
  accuracy.append(acc)
  print(f"Accuracy in fold{i} : ",acc*100)
  a =prec_rec(x_test2, y_test, tree)
  p.append(a[0])
  r.append(a[1])

  i+=1

print()

print("Mean accuracy  : ",np.mean(accuracy)*100)
print("Mean precision : ",np.mean(p)*100)
print("Mean recall    : ",np.mean(r)*100)


In [ ]:
# for DecisionTreeClassifier of sklearn
from sklearn.metrics import precision_score, recall_score

accuracy=[]
p=[]
r=[]
i=1
for itrain,itest in x.split(final_data):
  x_train  = data[itrain]
  x_test = data[itest]
  y_train = y[itrain]
  y_test =  y[itest]

  model3 = DecisionTreeClassifier(criterion = "entropy",min_samples_split = 10,random_state = 42)
  model3.fit(x_train, y_train)
  y_pred = model3.predict(x_test)

  acc = accuracy_score(y_test, y_pred)
  accuracy.append(acc)
  precision=precision_score(y_test, y_pred, average='macro')
  recall=recall_score(y_test, y_pred, average='macro')
  p.append(precision)
  r.append(recall)

  print(f"Accuracy in fold{i} : ",acc*100)
  i+=1
print()
print("Mean accuracy  : ",np.mean(accuracy)*100)
print("Mean precision : ",np.mean(p)*100)
print("Mean recall    : ",np.mean(r)*100)

After training each decision tree, perform reduced-error pruning to avoid overfitting. Use a
held-out validation set (20% of the training data) and iteratively prune nodes if the pruning does
not decrease the validation accuracy. For each non-leaf node, replace it with a leaf node whose
label is the most frequent class among its samples, and keep the change if validation accuracy is
maintained or improved. Continue until no further pruning improves validation accuracy.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(final_data, y, test_size=0.2, random_state=42)
print(len(x_train),len(y_train),len(x_test),len(y_test))


In [ ]:
import numpy as np
import pandas as pd
import copy
from sklearn.model_selection import train_test_split

def prune2(tree, x_test, y_test):
    flag = True
    while flag:
        flag = False
        nl_nodes = nonleafnode(tree)
        treecopy = copy.deepcopy(tree)
        bestacc = findaccuracy(x_test, y_test, tree)

        for node_path in nl_nodes:
            ptree = prunetree2(copy.deepcopy(tree), node_path, x_test, y_test)
            acc = findaccuracy(x_test, y_test, ptree)
            if acc >= bestacc:
                bestacc = acc
                treecopy = ptree
                flag = True

        tree = treecopy
    return tree

In [ ]:
def prunetree2(tree, npath, x_test, y_test):
    ntree = tree
    subtree = ntree
    for i in range(len(npath)-1):
        feature, val = npath[i]
        subtree = subtree[feature][val]

    feature, val = npath[-1]
    finalsubtree = subtree[feature][val]

    #labels
    labels = []
    collect_labels(finalsubtree, labels)

    if len(labels) == 0:
        return ntree
    temp=set(labels)
    mxlabel = max(temp, key=labels.count)

    subtree[feature][val] = mxlabel

    return ntree

In [ ]:
def nonleafnode(tree, path=()):
    v = []
    if isinstance(tree, dict):
        feature = list(tree.keys())[0]

        for i,s in tree[feature].items():
            if isinstance(s, dict):
                v.append(path + ((feature, i),))
                v.extend(nonleafnode(s, path + ((feature, i),)))
    return v


def collect_labels(tree, labels):
    if isinstance(tree, dict):
        feature = list(tree.keys())[0]
        for v, s in tree[feature].items():
            collect_labels(s, labels)
    else:
        labels.append(tree)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(final_data, y, test_size=0.2, random_state=42)
y_train2=pd.Series(y_train)
y_test2=pd.Series(y_test)
x_test2=pd.DataFrame(x_test)
tree = decision_tree_infogain(x_train, y_train2)
acc = findaccuracy(x_test2, y_test, tree)
print(f"Accuracy before pruning : ",acc*100)
print()
tree = prune2(tree, x_test, y_test2)
print("Tree before : ",tree)
# Accuracy check
acc = findaccuracy(x_test2, y_test, tree)
print()

print("Accuracy after pruning:", acc*100)
print()
print("Final tree:", tree)
